In [1]:
def smooth(a,WSZ=5):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)

    start = np.cumsum(a[:WSZ-1])[::2]/r
    #print start,out0
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

def digitalOnly(s):
    import string
    al=string.maketrans('','')
    nodigs=al.translate(al, string.digits)
    return s.translate(al, nodigs)

def getLinearRegressionCoef(X,y):
    import numpy as np
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    reg.score(X, y)    
    return reg.coef_


def computeDerivetive_od(t, sOD, wsz = 5):
    from numpy import isnan
    coef_OD = np.zeros(len(sOD))
    t_mean = np.zeros(len(sOD))
    j = 0
    nan_found = False
    for i in range(len(sOD)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]        
        sOD_wsz = np.expand_dims(np.log(sOD[i-wsz/2:i+wsz/2+1]),axis=1)    
        if len(sOD_wsz)!= wsz:
            coef_OD[i] = 0
        else:                
            if len(t_wsz)==len(sOD_wsz):                
                where_are_NaNs = isnan(sOD_wsz)
                if any(where_are_NaNs):
                    sOD_wsz[where_are_NaNs] = np.mean(sOD_wsz[~where_are_NaNs])
                    print "nan Found in array, replaced by mean value of the rest"

                coef_OD[i]=getLinearRegressionCoef(t_wsz,sOD_wsz)
                t_mean[i] = np.mean(t_wsz)
    
    dt = t_mean[wsz/2:len(t_mean)-wsz/2]
    dt_pad = np.pad(dt,[wsz/2,wsz/2],'reflect',reflect_type='odd')
    
    return coef_OD, dt_pad


def draw_tangent(x1,y1,ind,e=0.1,plot=False):
    from scipy import interpolate
    import numpy as np
    import matplotlib.pyplot as plt
    x = np.squeeze(x1)
    y = np.squeeze(y1)
    # interpolate the data with a spline
    spl = interpolate.splrep(x,y)
    a = x[ind]
    fa = interpolate.splev(a,spl,der=0)     # f(a)
    fprime = interpolate.splev(a,spl,der=1) # f'(a)
    left = 0
    for i in range(ind,0,-1):        
        a_i = x[i]
        y_i = y[i]        
        tan_i = fa+fprime*(a_i-a) # tangent                
        if abs(tan_i-y_i) > abs(y[ind]*e):
            left = i
            break    
    right = len(x) - 1
    for i in range(ind+1,len(x)):        
        a_i = x[i]
        y_i = y[i]
        tan_i = fa+fprime*(a_i-a) # tangent        
        if abs(tan_i-y_i) > abs(y[ind]*e):
            right = i
            break
    small_t = np.linspace(x[left],x[right],10)
    tan = fa+fprime*(small_t-a) # tangent    
    if plot:
        plt.plot(a,fa,'om',small_t,tan,'--r')
        plt.plot(x,y)
    return left, right,small_t,tan


def getTimepoints_old(coef_OD,sOD_log, dt_pad,start_ind, thres=0.03, thres_range=0.8):
    coef_OD_fromStart_ind = coef_OD[start_ind:]
    ind = np.argmax(coef_OD_fromStart_ind) + start_ind        
    max_v = coef_OD[ind]
    
    time_max_coefod = dt_pad[ind]        
    right_side_exp = len(coef_OD)-1
    left_side_exp = 0
    for k in range(ind+1,len(coef_OD)):
        v_od = coef_OD[k]
        if (max_v-v_od)/max_v > 1-thres_range:
            right_side_exp = k
            break

    for k in reversed(range(ind)):
        v_od = coef_OD[k]
        if (max_v-v_od)/max_v > 1-thres_range:
            left_side_exp = k
            break

    time_left = dt_pad[left_side_exp]
    time_right = dt_pad[right_side_exp]  
    
    return time_max_coefod,time_left,time_right,left_side_exp,right_side_exp

def getTimepoints(coef_OD,sOD_log,dt_pad,start_ind):
    coef_OD_fromStart_ind = coef_OD[start_ind:]
    ind = np.argmax(coef_OD_fromStart_ind) + start_ind        
    max_v = coef_OD[ind]
    time_max_coefod = dt_pad[ind]        
    left_side_exp, right_side_exp,small_t,tan = draw_tangent(dt_pad,sOD_log,ind)
    time_left = dt_pad[left_side_exp]
    time_right = dt_pad[right_side_exp]  
    return time_max_coefod,time_left,time_right,left_side_exp,right_side_exp,small_t,tan


def getPostiveStart(sOD):
    start_positive = 0
    for o in range(start_positive, len(sOD)):
        if sOD[o] <= 0:
            start_positive = o + 1
    if start_positive >= len(sOD):
        start_positive = -1
    return start_positive

def mkdir_p(path):
    import errno    
    import os
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
            
def getStationaryTime(sOD):
    sOD = np.asarray(sOD)
    sta_index = len(sOD)-1
    last_value = sOD[sta_index]
    for i in range(len(sOD)-2,0,-1):
        if (abs(last_value-sOD[i])/last_value)>0.05:
            sta_index = i
            #print "stationary phase found"
            break
    return sta_index

def mkdir_p(path):
    import errno    
    import os
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [10]:
import os
import pandas as pd

def getfiles(folderPath):
    pths = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and 'stats' not in fn:
                pth = os.path.join(root, fn)
                pths.append(pth)
    return pths

pths = getfiles('csv')

for path in pths:
    folder = path.split('.')[0]
    mkdir_p(folder)
    data = pd.read_csv(path)    
    
    import numpy as np
    import matplotlib.pyplot as plt
    import os
    time = data.iloc[:,0]
    m_od = data.iloc[0:,1:]
    strains = list(m_od.columns)
    t = time.values

    t_k = (t[1:len(t)] - t[0:len(t)-1])/3600
    t_k = np.cumsum(t_k)
    t_k = np.insert(t_k,0,0)
    t_k = t_k.reshape(-1,1)
    t = t_k

    blank_od = 0.04
    pre_max_v = 0.7

    growth_rate_dict = dict()
    lag_phase_dict = dict()
    yield_dict = dict()
    for name in strains:
        od = m_od[name].values
        od_array = od - np.min(blank_od)
        od_array = np.squeeze(np.asarray(od_array))
        sOD = smooth(od_array)    
        sOD_log = np.log(sOD)

        coef_OD, dt_pad = computeDerivetive_od(t, sOD)
        ind_max_dODdt = np.argmax(coef_OD)
        OD_exp = sOD[ind_max_dODdt]
        time_max_dODdt = dt_pad[ind_max_dODdt]
        start_positive = getPostiveStart(sOD)
        if start_positive > 0:
            print fn,'found negative od ', start_positive    
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/pre_max_v
        m = t < lag_phase_pre
        start_ind = np.sum(m)

        time_max_od,time_left, time_right,left_side_exp,right_side_exp,small_t,tan = getTimepoints(coef_OD,sOD_log,dt_pad,start_ind)

        time_max_dODdt = dt_pad[ind_max_dODdt]

        od_max_coef = coef_OD[ind_max_dODdt]

        ind = np.argmax(coef_OD)        
        max_v = coef_OD[ind]    
        growth_rate = max_v

        # Update lag_phase using growth rate in an estimated window
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/max_v
        lag_phase = lag_phase_pre

        sta_index = getStationaryTime(sOD)
        t_sta = dt_pad[sta_index]

        yield_mean = np.mean(sOD[sta_index:])
        print name+'yield'+"="+str(yield_mean)


        x = t_k
        y = sOD_log
        plt.plot(x,y,label='log(OD)')
        plt.plot(x,y,small_t,tan,'--r',label='log(OD)')
        plt.title(name)
        plt.xlabel("time")
        w = coef_OD
        plt.plot(x, w,label='dOD/dt/OD') # can c                
        plt.axvline(x=time_max_od,linestyle=':',color='green')
        plt.legend()
        plt.savefig(os.path.join(folder,name +'-dODdtOD.png'))
        plt.close()   

        plt.plot(x,sOD,label='OD')    
        plt.axvline(x=time_left,linestyle=':',color='red')
        plt.axvline(x=time_max_od,linestyle=':',color='green')
        plt.axvline(x=t_sta,linestyle=':',color='yellow')
        plt.axvline(x=time_right,linestyle=':',color='red')
        plt.axvline(x=lag_phase,linestyle=':',color='purple')    
        plt.xlabel('time')
        plt.title(name)        
        plt.legend()
        plt.savefig(os.path.join(folder,name +'-OD.png'))
        plt.close()

        growth_rate_dict[name] = growth_rate
        lag_phase_dict[name] = lag_phase
        yield_dict[name] = yield_mean

    df = pd.DataFrame.from_dict([growth_rate_dict,lag_phase_dict, yield_dict])
    df = df.transpose()
    df2 = df.reindex(strains)
    df2.columns = ['Growth Rate', 'Lag Phase', 'yield']
    df2.to_csv(folder +'-stats.csv')
    

alryield=0.39226238095238103
alr.1yield=0.4148306666666667
alr.2yield=0.40272864583333334
alr.3yield=0.40975190476190476
alr.4yield=0.4050523333333334
alr.5yield=0.40559493333333335
alr.6yield=0.39551231746031745
alr.7yield=0.41720654088050324
cbpMyield=0.3963681481481482
cbpM.1yield=0.39944746177370033
cbpM.2yield=0.3872828828828829
cbpM.3yield=0.39276716981132076
cbpM.4yield=0.3835097959183673
cbpM.5yield=0.39063980582524277
cbpM.6yield=0.3799584761904762
cbpM.7yield=0.39283512027491413
fixCyield=0.4117140251572327
fixC.1yield=0.4194347194719471
fixC.2yield=0.4123787333333333
fixC.3yield=0.41422803921568635
fixC.4yield=0.4071060377358491
fixC.5yield=0.4161918439716313
fixC.6yield=0.4050450526315789
fixC.7yield=0.42818865900383146
hybGyield=0.40848237547892713
hybG.1yield=0.4152828431372549
hybG.2yield=0.4060971193415638
hybG.3yield=0.4154401826484019
hybG.4yield=0.4016774561403509
hybG.5yield=0.3980901111111111
hybG.6yield=0.38073779874213837
hybG.7yield=0.41159871794871794
hycAyield

flu.4yield=0.27984712121212124
flu.5yield=0.27426136363636366
flu.6yield=0.27831633986928106
flu.7yield=0.2755418055555556
fruByield=0.30171914529914534
fruB.1yield=0.2847810526315789
fruB.2yield=0.29145111111111116
fruB.3yield=0.29295042735042737
fruB.4yield=0.2927074603174603
fruB.5yield=0.2870908527131783
fruB.6yield=0.2858172093023256
fruB.7yield=0.28442181818181816
fsRyield=0.3158271604938272
fsR.1yield=0.3174906666666667
fsR.2yield=0.31619
fsR.3yield=0.31115253333333337
fsR.4yield=0.3139016666666667
fsR.5yield=0.3191841975308642
fsR.6yield=0.31376345679012346
fsR.7yield=0.3177760493827161
ssuAyield=0.3154926666666667
ssuA.1yield=0.31183488888888894
ssuA.2yield=0.31605096774193553
ssuA.3yield=0.31239822222222224
ssuA.4yield=0.31688712643678163
ssuA.5yield=0.3083787096774194
ssuA.6yield=0.3144030107526882
ssuA.7yield=0.3097713978494624
WTyield=0.2786882269503546
WT.1yield=0.2660671428571429
WT.2yield=0.27283451851851853
WT.3yield=0.26431367521367527
WT.4yield=0.2693007246376812
WT.

sgcQ.6yield=0.3116624761904762
sgcQ.7yield=0.29686876190476197
ulaDyield=0.27234140350877195
ulaD.1yield=0.26970342342342346
ulaD.2yield=0.2683421621621621
ulaD.3yield=0.26923676190476187
ulaD.4yield=0.27121982905982917
ulaD.5yield=0.26620829268292684
ulaD.6yield=0.26985846153846155
ulaD.7yield=0.26788596491228067
uxuRyield=0.271531891891892
uxuR.1yield=0.25669962962962967
uxuR.2yield=0.25743025641025646
uxuR.3yield=0.24752752136752137
uxuR.4yield=0.2441070175438596
uxuR.5yield=0.246541981981982
uxuR.6yield=0.2519938211382114
uxuR.7yield=0.2544240350877193
WTyield=0.28612578947368417
WT.1yield=0.2754784761904762
WT.2yield=0.2760496296296297
WT.3yield=0.27160888888888896
WT.4yield=0.27134432432432437
WT.5yield=0.2616403603603604
WT.6yield=0.262774358974359
WT.7yield=0.26482203703703705
thiQyield=0.36233418803418804
thiQ.1yield=0.36205444444444446
thiQ.2yield=0.3407530081300813
thiQ.3yield=0.34228083333333337
thiQ.4yield=0.3279953333333333
thiQ.5yield=0.33560092592592594
thiQ.6yield=0.33

WT.7yield=0.2905650000000001
Paacyield=0.18319962962962963
Paac.1yield=0.17412333333333335
Paac.2yield=0.17377797101449274
Paac.3yield=0.17187523809523808
Paac.4yield=0.17311473684210527
Paac.5yield=0.16720333333333331
Paac.6yield=0.17760444444444443
Paac.7yield=0.1674776470588235
WTyield=0.3434017204301075
WT.1yield=0.3189693333333334
WT.2yield=0.3251472072072072
WT.3yield=0.31499809523809524
WT.4yield=0.3179238095238095
WT.5yield=0.3072616666666667
WT.6yield=0.31020720720720724
WT.7yield=0.30958824561403514
yajQyield=0.36266495726495723
yajQ.1yield=0.3263607142857143
yajQ.2yield=0.36298837606837614
yajQ.3yield=0.3053098039215686
yajQ.4yield=0.3049288288288288
yajQ.5yield=0.29742407407407406
yajQ.6yield=0.2987099099099099
yajQ.7yield=0.2990899047619048
ybityield=0.34706740740740744
ybit.1yield=0.34016839506172836
ybit.2yield=0.32828000000000007
ybit.3yield=0.32335812500000005
ybit.4yield=0.3235996296296296
ybit.5yield=0.31268392156862745
ybit.6yield=0.3129688288288288
ybit.7yield=0.31

melA.2yield=0.2690351020408164
melA.3yield=0.26614782608695653
melA.4yield=0.2713093617021277
melA.5yield=0.2635077777777778
melA.6yield=0.26503032679738564
melA.7yield=0.25651791666666673
sydyield=0.2581622222222222
syd.1yield=0.2551556
syd.2yield=0.25671607843137256
syd.3yield=0.2571371428571429
syd.4yield=0.25349084967320257
syd.5yield=0.25618730769230774
syd.6yield=0.25393129251700686
syd.7yield=0.2564337681159421
WTyield=0.27241771929824565
WT.1yield=0.2774753333333334
WT.2yield=0.28996030303030307
WT.3yield=0.2792050793650794
WT.4yield=0.27701076923076934
WT.5yield=0.26539298245614035
WT.6yield=0.28655982905982913
WT.7yield=0.30278108527131786
ybaJyield=0.26502025157232706
ybaJ.1yield=0.2656478666666667
ybaJ.2yield=0.2641801333333334
ybaJ.3yield=0.2668282269503546
ybaJ.4yield=0.26145920000000006
ybaJ.5yield=0.2620565277777778
ybaJ.6yield=0.26310052287581703
ybaJ.7yield=0.27323362962962966
ydjKyield=0.27039006802721094
ydjK.1yield=0.26918277777777777
ydjK.2yield=0.2730898550724637

# Load csv 

In [31]:
import os
import pandas as pd

def getfiles(folderPath):
    pths = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and 'stats' in fn:
                pth = os.path.join(root, fn)
                pths.append(pth)
    return pths

pths = getfiles('csv')

extrac_gr = "Growth Rate"
extrac_yi = "yield"

yield_list = []
growth_rate_list = []
name_list = []
for path in pths:
    print path
    data = pd.read_csv(path)
    data2 = data.set_index(data.columns[0])
    pre_strain ='no'
    for strain in data2.index:
        if pre_strain not in strain:
            try:
                if pre_strain != 'no':
                    print "s3",pre_strain
                    yield_list.append(yield_dict)
                    growth_rate_list.append(growth_dict)
                    name_list.append(pre_strain)
            except:
                print pre_strain,strain

        if pre_strain != strain:
            if pre_strain not in strain:
                pre_strain = strain
                #name_list.append(pre_strain)
                #print "s1",pre_strain, strain
                i = 1
                yield_dict = dict()
                growth_dict = dict()

        init = 'R'+str(i)
        if pre_strain in strain:
            #print "s2",pre_strain, strain
            #print init,data2.loc[strain,extrac]
            yield_dict[init] = data2.loc[strain,extrac_yi]
            growth_dict[init] = data2.loc[strain, extrac_gr]
            i = i +1


print "last s3",pre_strain
yield_list.append(yield_dict)
growth_rate_list.append(growth_dict)
name_list.append(pre_strain)


csv/MG1655 10082019 N-stats.csv
s2 agaV agaV
s2 agaV agaV.1
s2 agaV agaV.2
s2 agaV agaV.3
s2 agaV agaV.4
s2 agaV agaV.5
s2 agaV agaV.6
s2 agaV agaV.7
s3 agaV
s2 gltK gltK
s2 gltK gltK.1
s2 gltK gltK.2
s2 gltK gltK.3
s2 gltK gltK.4
s2 gltK gltK.5
s2 gltK gltK.6
s2 gltK gltK.7
s3 gltK
s2 gspF gspF
s2 gspF gspF.1
s2 gspF gspF.2
s2 gspF gspF.3
s2 gspF gspF.4
s2 gspF gspF.5
s2 gspF gspF.6
s2 gspF gspF.7
s3 gspF
s2 mtlD mtlD
s2 mtlD mtlD.1
s2 mtlD mtlD.2
s2 mtlD mtlD.3
s2 mtlD mtlD.4
s2 mtlD mtlD.5
s2 mtlD mtlD.6
s2 mtlD mtlD.7
s3 mtlD
s2 rarD rarD
s2 rarD rarD.1
s2 rarD rarD.2
s2 rarD rarD.3
s2 rarD rarD.4
s2 rarD rarD.5
s2 rarD rarD.6
s2 rarD rarD.7
s3 rarD
s2 rsd rsd
s2 rsd rsd.1
s2 rsd rsd.2
s2 rsd rsd.3
s2 rsd rsd.4
s2 rsd rsd.5
s2 rsd rsd.6
s2 rsd rsd.7
s3 rsd
s2 uxaB uxaB
s2 uxaB uxaB.1
s2 uxaB uxaB.2
s2 uxaB uxaB.3
s2 uxaB uxaB.4
s2 uxaB uxaB.5
s2 uxaB uxaB.6
s2 uxaB uxaB.7
s3 uxaB
s2 WT WT
s2 WT WT.1
s2 WT WT.2
s2 WT WT.3
s2 WT WT.4
s2 WT WT.5
s2 WT WT.6
s2 WT WT.7
s3 WT
s2 yagS yag

s2 caiB caiB
s2 caiB caiB.1
s2 caiB caiB.2
s2 caiB caiB.3
s2 caiB caiB.4
s2 caiB caiB.5
s2 caiB caiB.6
s2 caiB caiB.7
s3 caiB
s2 gspM gspM
s2 gspM gspM.1
s2 gspM gspM.2
s2 gspM gspM.3
s2 gspM gspM.4
s2 gspM gspM.5
s2 gspM gspM.6
s2 gspM gspM.7
s3 gspM
s2 melA melA
s2 melA melA.1
s2 melA melA.2
s2 melA melA.3
s2 melA melA.4
s2 melA melA.5
s2 melA melA.6
s2 melA melA.7
s3 melA
s2 syd syd
s2 syd syd.1
s2 syd syd.2
s2 syd syd.3
s2 syd syd.4
s2 syd syd.5
s2 syd syd.6
s2 syd syd.7
s3 syd
s2 WT WT
s2 WT WT.1
s2 WT WT.2
s2 WT WT.3
s2 WT WT.4
s2 WT WT.5
s2 WT WT.6
s2 WT WT.7
s3 WT
s2 ybaJ ybaJ
s2 ybaJ ybaJ.1
s2 ybaJ ybaJ.2
s2 ybaJ ybaJ.3
s2 ybaJ ybaJ.4
s2 ybaJ ybaJ.5
s2 ybaJ ybaJ.6
s2 ybaJ ybaJ.7
s3 ybaJ
s2 ydjK ydjK
s2 ydjK ydjK.1
s2 ydjK ydjK.2
s2 ydjK ydjK.3
s2 ydjK ydjK.4
s2 ydjK ydjK.5
s2 ydjK ydjK.6
s2 ydjK ydjK.7
s3 ydjK
s2 yhbW yhbW
s2 yhbW yhbW.1
s2 yhbW yhbW.2
s2 yhbW yhbW.3
s2 yhbW yhbW.4
s2 yhbW yhbW.5
s2 yhbW yhbW.6
s2 yhbW yhbW.7
s3 yhbW
s2 yhcM yhcM
s2 yhcM yhcM.1
s2 yhcM yhcM.2


s2 Blank Blank
s2 Blank Blank.1
s2 Blank Blank.2
s2 Blank Blank.3
s2 Blank Blank.4
s2 Blank Blank.5
s2 Blank Blank.6
s2 Blank Blank.7
s3 Blank
s2 yjfC yjfC
s2 yjfC yjfC.1
s2 yjfC yjfC.2
s2 yjfC yjfC.3
s2 yjfC yjfC.4
s2 yjfC yjfC.5
s2 yjfC yjfC.6
s2 yjfC yjfC.7
s3 yjfC
s2 yhbS yhbS
s2 yhbS yhbS.1
s2 yhbS yhbS.2
s2 yhbS yhbS.3
s2 yhbS yhbS.4
s2 yhbS yhbS.5
s2 yhbS yhbS.6
s2 yhbS yhbS.7
s3 yhbS
s2 ssuE ssuE
s2 ssuE ssuE.1
s2 ssuE ssuE.2
s2 ssuE ssuE.3
s2 ssuE ssuE.4
s2 ssuE ssuE.5
s2 ssuE ssuE.6
s2 ssuE ssuE.7
s3 ssuE
s2 yhcM yhcM
s2 yhcM yhcM.1
s2 yhcM yhcM.2
s2 yhcM yhcM.3
s2 yhcM yhcM.4
s2 yhcM yhcM.5
s2 yhcM yhcM.6
s2 yhcM yhcM.7
s3 yhcM
s2 yafV yafV
s2 yafV yafV.1
s2 yafV yafV.2
s2 yafV yafV.3
s2 yafV yafV.4
s2 yafV yafV.5
s2 yafV yafV.6
s2 yafV yafV.7
s3 yafV
s2 napC napC
s2 napC napC.1
s2 napC napC.2
s2 napC napC.3
s2 napC napC.4
s2 napC napC.5
s2 napC napC.6
s2 napC napC.7
s3 napC
s2 ybbP ybbP
s2 ybbP ybbP.1
s2 ybbP ybbP.2
s2 ybbP ybbP.3
s2 ybbP ybbP.4
s2 ybbP ybbP.5
s2 ybbP ybbP.

In [29]:
strain

'WT.7'

In [23]:
df = pd.DataFrame(yield_list) 
df = df.set_index([pd.Index(name_list)])
df
df.to_csv('yield_all_strains.csv')

df = pd.DataFrame(growth_rate_list) 
df = df.set_index([pd.Index(name_list)])
df
df.to_csv('growth_rate_all_strains.csv')

# run for one file .csv

In [ ]:
import pandas as pd 
import os
fn = "MG1655 05082019.csv"
folder = fn.split('.')[0]
mkdir_p(folder)
data = pd.read_csv("MG1655 05082019.csv") 
# Preview the first 5 lines of the loaded data 
#data.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
time = data['Time [s]']
m_od = data.iloc[0:,1:]
strains = list(m_od.columns)
t = time.values

t_k = (t[1:len(t)] - t[0:len(t)-1])/3600
t_k = np.cumsum(t_k)
t_k = np.insert(t_k,0,0)
t_k = t_k.reshape(-1,1)
t = t_k

blank_od = 0.04
pre_max_v = 0.7

growth_rate_dict = dict()
lag_phase_dict = dict()
yield_dict = dict()
for name in strains:
    od = m_od[name].values
    od_array = od - np.min(blank_od)
    od_array = np.squeeze(np.asarray(od_array))
    sOD = smooth(od_array)    
    sOD_log = np.log(sOD)

    coef_OD, dt_pad = computeDerivetive_od(t, sOD)
    ind_max_dODdt = np.argmax(coef_OD)
    OD_exp = sOD[ind_max_dODdt]
    time_max_dODdt = dt_pad[ind_max_dODdt]
    start_positive = getPostiveStart(sOD)
    if start_positive > 0:
        print fn,'found negative od ', start_positive    
    lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/pre_max_v
    m = t < lag_phase_pre
    start_ind = np.sum(m)

    time_max_od,time_left, time_right,left_side_exp,right_side_exp,small_t,tan = getTimepoints(coef_OD,sOD_log,dt_pad,start_ind)

    time_max_dODdt = dt_pad[ind_max_dODdt]

    od_max_coef = coef_OD[ind_max_dODdt]

    ind = np.argmax(coef_OD)        
    max_v = coef_OD[ind]    
    growth_rate = max_v
    
    # Update lag_phase using growth rate in an estimated window
    lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/max_v
    lag_phase = lag_phase_pre

    sta_index = getStationaryTime(sOD)
    t_sta = dt_pad[sta_index]
    
    yield_mean = np.mean(sOD[sta_index:])
    print name+'yield'+"="+str(yield_mean)
    
        
    x = t_k
    y = sOD_log
    plt.plot(x,y,label='log(OD)')
    plt.plot(x,y,small_t,tan,'--r',label='log(OD)')
    plt.title(name)
    plt.xlabel("time")
    w = coef_OD
    plt.plot(x, w,label='dOD/dt/OD') # can c                
    plt.axvline(x=time_max_od,linestyle=':',color='green')
    plt.legend()
    plt.savefig(os.path.join(folder,name +'-dODdtOD.png'))
    plt.close()   

    plt.plot(x,sOD,label='OD')    
    plt.axvline(x=time_left,linestyle=':',color='red')
    plt.axvline(x=time_max_od,linestyle=':',color='green')
    plt.axvline(x=t_sta,linestyle=':',color='yellow')
    plt.axvline(x=time_right,linestyle=':',color='red')
    plt.axvline(x=lag_phase,linestyle=':',color='purple')    
    plt.xlabel('time')
    plt.title(name)        
    plt.legend()
    plt.savefig(os.path.join(folder,name +'-OD.png'))
    plt.close()
    
    growth_rate_dict[name] = growth_rate
    lag_phase_dict[name] = lag_phase
    yield_dict[name] = yield_mean
    
df = pd.DataFrame.from_dict([growth_rate_dict,lag_phase_dict, yield_dict])
df = df.transpose()
df2 = df.reindex(strains)
df2.columns = ['Growth Rate', 'Lag Phase', 'yield']
df2.to_csv(os.path.join('.',name +'-stats.csv'))    
    